# OpenStreetMap Data Wrangling Project
### Sunny Jiang 03/01/2017

## Map Area

Chicago, Illinois, United States

I lived in Chicago for 3 years when I was in graduate school and still visit it on regular basis today. So I'm interested to dive into this dataset and test some queries.

I downloaded the Chicago data extracts from [Mapzen](https://mapzen.com/data/metro-extracts/metro/chicago_illinois/) last year. But seems like it's shutting down its services. So the link doesn't work anymore... Here is the link to [OpenStreetMap](https://www.openstreetmap.org/export#map=9/41.6616/-86.9101) where Overpass API is available. 

## Problems Encountered in the Map

I first created a sample data file using 100-th top level element from the entire Chicago data set. From the initial explore, I noticed:

- Overabbreviated street names (*"Walker Ave", "Randall RD"*)


### Overabbreviated street names

I created a **update_name** function, then use **shape_element** function looping through the tags of nodes and ways. When the key is street, check whether the value meets the expected words. If not, updating the value using the mapping list.

This updated all substrings, such that: *"Walker Ave"* becomes *"Walker Avenue"* and *"Randall RD"* becomes *"Randall Road"*

```python
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]
mapping = { "St": "Street",
            "St.": "Street",
            "Ave" : "Avenue",
            "Ave." : "Avenue",
            "Rd" : "Road",
            "Rd." : "Road",
            "RD" : "Road",
            "Blvd" : "Boulevard",
            "Blvd." : "Boulevard",
            "Cir" : "Circle",
            "Cir." : "Circle",
            "Ct" : "Court",
            "Ct." : "Court",
            "Dr" : "Drive",
            "Dr." : "Drive",
            "Pl" : "Place",
            "Pl." : "Place"
            }

def update_name(name, mapping):
    m = street_type_re.search(name)
    other_street_types = []
    if m:
        street_type = m.group()
        if street_type in mapping.keys():
            name = re.sub(street_type_re,mapping[street_type],name)
        else:
            other_street_types.append(street_type)
    return name

def shape_element ...
    if tag.attrib['k'] == "addr:street":
        node_tags['id'] = element.attrib['id']
        node_tags['key'] = 'street'
        node_tags['type'] = "addr"
        node_tags['value'] = update_name(tag.attrib['v'], mapping)
        tags.append(node_tags.copy())
```

## Overview of the Data

### File Sizes

- chicago_illinois.osm ---------- 2.1GB
- nodes.csv --------------------- 755MB
- nodes_tags.csv ---------------- 19MB
- ways.csv ---------------------- 79MB
- ways_tags.csv ----------------- 220MB
- ways_nodes.csv ---------------- 243MB

### Number of Nodes & Ways

``` python
select count(*) from nodes;  --8,792,245
select count(*) from ways;   --1,249,804
```

### Number of Users

There are 2936 users contributed to this data set. "chicago-buildings" alone contributed more than 55% of the data. While 662 of the users only contributed once.

```python
-- distinct users for nodes and ways

with cte as
(select id, user_  from nodes
union all
select id, user_  from ways)
select count(distinct user_) from cte; -- 2,936

-- sort the users by the contribute frequency

with cte as
(select id, user_  from nodes
union all
select id, user_  from ways)
select distinct user_, count(*) from cte
group by user_
order by count(*) desc;
```

<img src="top users.png">

### Type of cuisines

Pizza and Mexican are the most popular type of cuisines!

``` python
with cte as
(SELECT distinct id FROM NODES_TAGS
where key = 'amenity'
and value = 'restaurant')
select value, count(*) from nodes_tags a
join cte b
on a.id = b.id
where key = 'cuisine'
group by value
order by count(*) desc;
```

<img src="cuisine.png">

## Other Ideas about the Dataset

### Postcode & Geographic Data

Top 2 postcodes are from Evanston IL which is what we expected. While if we sort the postcode, we can see we have some postcodes actually belong to Indiana.


``` python
-- postcode by frequency

select distinct value, count(*) from nodes_tags
where key = 'postcode'
group by value
order by count(*) desc;

-- inconsistent postcodes

select distinct value, count(*) from nodes_tags
where key = 'postcode'
group by value
order by value;
```

<tr>
<td> <img src="postcode frequency.png">
<td> <img src="postcode wrong.png">
</tr>

So I used Tableau to plot the postcode and also latitute and longitute. From the first image we can see we have a few problematic postcodes in other parts of US. Zoom in and we can see a few more in Indiana. 

Then I ploted the Latitude and Longitude data of nodes. Clearly we can see this data set is filtered by geografic information!

<tr>
<td> <img src="tableau1.png">
<td> <img src="tableau2.png">
<td> <img src="tableau3.png">
</tr>

Find the Min and Max of LAT & LON from nodes data. Then highlight that range in OpenStrreMap. We can see it's not only included Chicago but also a north west corner of Indiana and a few outliers in the middle of Michigan Lake. If we could just reduce the minimum Longitude from -87.066 to -87.528, the state border between Illinois and Indiana, we can greatly increase the Chicago data quanlity.

<img src="lat long.png">

<tr>
<td> <img src="current lat lon.png">
<td> <img src="new lat lon.png">
</tr>